In [1]:
using DualNumbers, LaTeXStrings, Plots
pyplot()

Plots.PyPlotBackend()

In [2]:
"""
    ciclosestables!(xx, f, nit, nout, cc)

Esta función itera el mapeo `f`, de una variable, `nit+nout` veces, 
usando como condición inicial `x0=0`; los últimos `nout` iterados 
actualizan al vector `xx` que tiene longitud `nout`. `cc` es el valor
del parámetro del mapeo `f`. El mapeo `f` debe ser definido de 
tal manera que `f(x0,cc)` tenga sentido. La idea es los últimos 
`nout` iterados reflejen los ciclos estables del mapeo `f`. 
"""
function ciclosestables!(xx, f, nit, nout, cc)
    @assert nit > 0 && nout > 0
    
    # Primeros nit iterados
    x0 = 0.0
    for it = 1:nit
        x0 = f(x0, cc)
    end
    
    # Se guardan los siguientes nout iterados
    for it = 1:nout
        x0 = f(x0, cc)
        @inbounds xx[it] = x0
    end
    
    return nothing
end

"""
    diagbifurc(f, nit, nout, crange)

Itera el mapeo `f` `nit+nout` veces y regresa una matriz
cuya columna `i` tiene los últimos `nout` iterados del mapeo
para el valor del parámetro del mapeo `crange[i]`.

La función `f` debe ser definida de tal manera que `f(x0, c)` 
tenga sentido.
"""
function diagbifurc(f, nit, nout, crange)
    xx = Vector{Float64}(nout)
    #Lo voy a guardar en un array.
    ff = Array{Float64,2}(nout, length(crange))
    #ff = Any[]
    
    for ic in eachindex(crange)
        c = crange[ic]
        ciclosestables!(xx, f, nit, nout, c)
        
        #push!(ff, xx)
        
        ff[:,ic] = xx
        
    end
    
    return ff
end

diagbifurc (generic function with 1 method)

In [3]:
Qc(x,c) = x^2 + c

crange = 0.25:-1/2^10:-1.0

ff = diagbifurc(Qc, 1000, 256, crange); 
cc = ones(size(ff)[1])*crange';

# Esto cambia las matrices en vectores; ayuda un poco para los dibujos
#ff = reshape(ff, size(ff)[1]*size(ff)[2]);
#cc = reshape(cc, size(ff));

In [4]:
figure(figsize=(6,4))
plot(cc, ff, "b,")
plot([-1.2,-1.5,-1.5,-1.2,-1.2],[-1.5,-1.5,-0.9,-0.9,-1.5], "k-")
plot([-1,0.5],[0.0,0.0], "r-")
xlabel(L"c")
ylabel(L"x_\infty")
title("Fig. 1")

LoadError: LoadError: UndefVarError: figure not defined
while loading In[4], in expression starting on line 1

-----

In [5]:
function find_one_bif(ff, cc)

    Resta = []
    bif = 0

    for i in 1:length(ff[1,:])

        resta = abs(ff[1, i] - ff[2, i])

        if resta > 0.0001

            bif = i
            break

        end

    end

    bif, cc[1, bif]

end

find_one_bif (generic function with 1 method)

In [6]:
find_one_bif(f1, c1) #funciona! Aunque está bien chafa.

LoadError: LoadError: UndefVarError: f1 not defined
while loading In[6], in expression starting on line 1

#### voy a tratar de refinar las orbitas usando métdo de newton

Es necesario tener una función que te genera la función iterada en sí misma. Entonces...

### Metaprogramming (cuadrica y más allá...)

In [7]:
nombre(n::Int) = symbol( string("F_", n) )

nombre (generic function with 1 method)

In [8]:
function itera_funcion(n)
    
    x = "x^2 + c"

    for i in 1:n-1

        x = "($x)^2 +c"

    end

    ex = parse(x)
    ex_ret = :( $(nombre(n))(x, c) = $ex )
    ex_ret
end 

itera_funcion (generic function with 1 method)

In [9]:
itera_funcion_identidad(1)

LoadError: LoadError: UndefVarError: itera_funcion_identidad not defined
while loading In[9], in expression starting on line 1

Funciona con `DualNumbers` y está muy chido!

### Adecuando `compute_roots`

Lo que busco es darle de comer las orbitas de la función de arriba para que la refine

In [10]:
"""
    compute_roots(f::Function, xx, c)
    Out: roots

xx is an array, cc is the parameter, f is the funtion

"""
function compute_roots(f::Function, A, cc)
    
    roots = similar(A)

    for j in 1:length(A[1,:])
        
        c = cc[1, j]
        
        for i in 1:length(A[:,1])
            
            xi = Dual(A[i, j], 1)

            # 100 iterations of Newton's method
            for k in 1:100
                x_2 = realpart(xi) - (realpart(f(xi, c)) - realpart(xi)) / (dualpart(f(xi, c)) - 1) 
                xi = Dual(x_2, 1)
            end

            roots[i, j] = realpart(xi)
        end
    end
    
    roots
end

compute_roots (generic function with 1 method)

In [11]:
function delete_equals(A)

    for j  in 1:length(A)
        for i  in 1:length(A)

            if i == j
                
                nothing

                elseif abs(abs(A[i]) - abs(A[j])) < 1e-5

                A[i] = NaN
                
            end
        end
    end
    
    deleteat!(A,find(isnan, A))
    
end

delete_equals (generic function with 1 method)

# Ahora integro todo lo que hice arriba al método de Luis

Pasos para acomplar las funciones:
1. `diagbifuc`: se crea una matriz con las orbitas de las iteraciones.
2. `computeroots`: refinamos las orbitas.
    - itera_funcion
    - Condiciones para aplicar la cuádrica.
    
3. Ver si poner los datos en una matriz o en un array.

--------

In [12]:
Qc(x,c) = x^2 + c

crange = 0.25:-1/2^10:-1.0

ff = diagbifurc(Qc, 10000, 1, crange); 
cc = ones(size(ff)[1])*crange';

In [13]:
ff

1x1281 Array{Float64,2}:
 0.4999  0.46875  0.455806  0.445873  0.4375  …  -0.998043  -0.999022  -1.0

In [14]:
eval(itera_funcion_identidad(2))

LoadError: LoadError: UndefVarError: itera_funcion_identidad not defined
while loading In[14], in expression starting on line 1

In [15]:
eval(itera_funcion_identidad(1))

LoadError: LoadError: UndefVarError: itera_funcion_identidad not defined
while loading In[15], in expression starting on line 1

In [16]:
ff_roots = compute_roots(F_2, ff, cc)

LoadError: LoadError: UndefVarError: F_2 not defined
while loading In[16], in expression starting on line 1

In [17]:
compute_roots(F_1, ff, cc)

LoadError: LoadError: UndefVarError: F_1 not defined
while loading In[17], in expression starting on line 1

> Ya quedó el método de newton listo para usarse. Abajo hago un método de newton que calcula la raíz de una sola condición inicial y no de toda la matriz.

In [18]:
"""
    compute_roots_paso(f::Function, x0, c)
"""
function compute_roots_paso(f::Function, x0, c)
            
            xi = Dual(x0, 1)

            # 1000 iterations of Newton's method
    for i in 1:1000

        x_2 = realpart(xi) - (realpart(f(xi, c)) - realpart(xi)) / (dualpart(f(xi, c)) - 1)
        xi = Dual(x_2, 1)
    end

    realpart(xi)
end

compute_roots_paso (generic function with 1 method)

In [19]:
ff

1x1281 Array{Float64,2}:
 0.4999  0.46875  0.455806  0.445873  0.4375  …  -0.998043  -0.999022  -1.0

In [20]:
for i in 1:length(ff)
    
    ff[i] = compute_roots_paso(F_2, ff[i], crange[i])
    
end

LoadError: LoadError: UndefVarError: F_2 not defined
while loading In[20], in expression starting on line 1

In [21]:
ff

1x1281 Array{Float64,2}:
 0.4999  0.46875  0.455806  0.445873  0.4375  …  -0.998043  -0.999022  -1.0

In [22]:
compute_roots_paso(F_1, ff[1], crange[1])

LoadError: LoadError: UndefVarError: F_1 not defined
while loading In[22], in expression starting on line 1

> El siguiente paso es evaluart el punto en $Q^p_c (x_0) = x_1$ y comparar $x_0$ y $x_i$ usando una toleracia. Si la distancia entre esos dos puntos es mayor a la tolerancia, es una bifurcación. 

# Algoritmo para encontrar bifurcaciones

- Se calcula una matriz $A : 1 \times n$, es decir sólo con el último punto de las orbitas calculadas.

Ahora se analiza indivialmente cada entrada de $A$:

- Aplicamos el método de Newton para el punto, obtenemos $x^*_i$
    - Si la orbita es periodo n, el método de Newton se aplica con $Q^{n+1}$
    

- Evaluamos ese punto resultante de nuevo el la función: $f(x^*_i) = x_\epsilon$.

    - Si $x^*_i = x_\epsilon$: no es bifurcación.
        
    - Si $|x^*_i - x_\epsilon| > \delta$: es una bifurcación.
    
        - Se empuja los valores de $c_i$ donde hay una bifurcación en un arreglo.
        
- Repetir para todo los valores del arreglo.

In [51]:
Qc(x,c) = x^2 + c

crange = 0.25:-1/2^10:-1.0

ff = diagbifurc(Qc, 10000, 1, crange); 
cc = ones(size(ff)[1])*crange';

In [52]:
cc

1x1281 Array{Float64,2}:
 0.25  0.249023  0.248047  0.24707  0.246094  …  -0.998047  -0.999023  -1.0

> para la primera entrada aplicamos en algoritmo descrito arriba, luego lo generalizo en un for loop.

In [53]:
ff[1]

0.49990011965147696

In [26]:
?compute_roots_paso

search: compute_roots_paso compute_roots



```
compute_roots_paso(f::Function, x0, c)
```


In [27]:
eval(itera_funcion(2))

F_2 (generic function with 1 method)

In [54]:
x_1 = compute_roots_paso(F_2, ff[1], cc[1])

0.4999999977223055

In [55]:
x_1 == Qc(x_1, cc[1])

true

$\implies$ no hay bifurcación para `cc[1]`.

In [30]:
Q1 = eval(itera_funcion(2))

F_2 (generic function with 1 method)

In [31]:
?Q1

search: Q1



No documentation found.

`Q1` is a generic `Function`.

```julia
# 1 method for generic function "F_2":
F_2(x, c) at In[8]:12
```


In [32]:
Q1(ff[1], cc[1])

0.49990013960165225

In [33]:
function albi_the_dragon(n, x, c)
    
    Fn = eval(itera_funcion(n))
    Fn(x, c)
    
end

albi_the_dragon (generic function with 1 method)

In [69]:
function find_bifurcation(FF, CC)
    
    bifurcaciones = Any[]
    eval(itera_funcion(1))
    n = 2
    
    for i in 1:length(FF)
        
        #Q_n = eval(itera_funcion(n))
        
        x_i = compute_roots_paso(eval(itera_funcion(n)), FF[i], CC[i])
      
        x_e = albi_the_dragon(n, x_i, CC[i])#F_1(x_i, CC[i])
        
        if abs(x_i - x_e) < 1e-10
            
            nothing
            
        elseif abs(x_i - x_e) > 1e-10
            
            n += 1
            push!(bifurcaciones, [CC[i], i])
            
        end
        
    end
    
    bifurcaciones, n
    
end     

find_bifurcation (generic function with 1 method)

In [70]:
crange = 0.25:-1/2^10:-1.0

ff = diagbifurc(Qc, 10000, 1, crange); 
cc = ones(size(ff)[1])*crange';

In [71]:
find_bifurcation(ff, cc)

(Any[],2)

In [96]:
crange = 0.25:-1/2^10:-1.0

ff = diagbifurc(Qc, 10000, 2, crange); 
cc = ones(size(ff)[1])*crange';

In [97]:
ff[:,1280]

2-element Array{Float64,1}:
 -0.999022   
 -0.000977518

In [39]:
abs(ff[1, 1280] - F_2(ff[1, 1280], cc[1,1280])) > 1e-10

false

In [40]:
a = ff[1, 1026]

-0.5312500000468743

In [41]:
b = F_2(ff[1, 1026], cc[1,1026])

-0.5312500000466912

In [42]:
a - b > 1e-10

false

In [43]:
f1 = itera_funcion(1)

:(F_1(x,c) = begin  # In[8], line 12:
            x ^ 2 + c
        end)

In [44]:
f1

:(F_1(x,c) = begin  # In[8], line 12:
            x ^ 2 + c
        end)

In [45]:
f1 = itera_funcion(2)

:(F_2(x,c) = begin  # In[8], line 12:
            (x ^ 2 + c) ^ 2 + c
        end)

In [46]:
f1

:(F_2(x,c) = begin  # In[8], line 12:
            (x ^ 2 + c) ^ 2 + c
        end)

In [47]:
function hola()
    
    eval(itera_funcion(10))
    F_10(0.0, 1.0)
end
    

hola (generic function with 1 method)

In [48]:
hola()

3.7918623102659254e90

In [49]:
F_10 

F_10 (generic function with 1 method)

In [50]:
albi_the_dragon(10, 0.0, 1.0)

3.7918623102659254e90

In [95]:
compute_roots_paso(eval(itera_funcion(3)), 0.5, -0.77)

1.5099504938362078